In [ ]:
import sqlite3

In [ ]:
anam = list(tuple())
anam.append(('철남', 3.6, '없음', '성북구 안암로 73-7'))
anam.append(('용초수', 4.3, '일요일', '성북구 고려대로27길 20'))
anam.append(('이공김밥', 4.1, '일요일', '성북구 안암로 61-6'))
anam.append(('고른햇살', 4.4, '없음', '성북구 개운사길 14'))
anam.append(('삼성통닭', 4.0, '첫째 셋째 일요일', '성북구 고려대로24길 60'))

In [ ]:
#테이블 생성 및 데이터 삽입
conn = sqlite3.connect('anam_restaurant.db')

# db 쿼리를 조작하기 위한 커서 객체 생성
cur = conn.cursor()

# 테이블 생성
cur.execute("""CREATE TABLE IF NOT EXISTS anam_food(
  id INTEGER PRIMARY KEY,
  name TEXT,
  rating REAL,
  closed_day TEXT,
  address TEXT
  )""")

cur.executemany('INSERT INTO anam_food (name, rating, closed_day, address) VALUES (?, ?, ?, ?)', anam)
conn.commit()

In [ ]:
#데이터 조회
conn = sqlite3.connect("anam_restaurant.db")

cur = conn.cursor()
cur.execute("SELECT * FROM anam_food")

Restaurant = cur.fetchall()

for restaurant in Restaurant:
  print(restaurant)

# conn.close()

(1, '철남', 3.6, '없음', '성북구 안암로 73-7')
(2, '용초수', 4.3, '일요일', '성북구 고려대로27길 20')
(3, '이공김밥', 4.1, '일요일', '성북구 안암로 61-6')
(4, '고른햇살', 4.4, '없음', '성북구 개운사길 14')
(5, '삼성통닭', 4.0, '첫째 셋째 일요일', '성북구 고려대로24길 60')


In [ ]:
conn = sqlite3.connect("anam_restaurant.db")

cur = conn.cursor()
cur.execute("DROP TABLE anam_food")


In [ ]:
import pandas as pd

conn = sqlite3.connect('anam_restaurant.db')
cur = conn.cursor()

cur.execute("SELECT * FROM anam_food")

rows = cur.fetchall()

# 테이블 컬럼명 가져오기
columns = [column[0] for column in cur.description]

frame = pd.DataFrame.from_records(data=rows, columns=columns)

conn.close()

frame

,id,name,rating,closed_day,address
0,1,철남,3.6,없음,성북구 안암로 73-7
1,2,용초수,4.3,일요일,성북구 고려대로27길 20
2,3,이공김밥,4.1,일요일,성북구 안암로 61-6
3,4,고른햇살,4.4,없음,성북구 개운사길 14
4,5,삼성통닭,4.0,첫째 셋째 일요일,성북구 고려대로24길 60


**Function Call**

In [ ]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.1 MB/s eta 0:00:00


In [ ]:
import sqlite3

conn = sqlite3.connect("anam_restaurant.db")

In [ ]:
import pandas as pd

conn = sqlite3.connect('anam_restaurant.db')
cur = conn.cursor()

cur.execute("SELECT * FROM anam_food")

rows = cur.fetchall()

# 테이블 컬럼명 가져오기
columns = [column[0] for column in cur.description]

frame = pd.DataFrame.from_records(data=rows, columns=columns)

frame

,id,name,rating,closed_day,address
0,1,철남,3.6,없음,성북구 안암로 73-7
1,2,용초수,4.3,일요일,성북구 고려대로27길 20
2,3,이공김밥,4.1,일요일,성북구 안암로 61-6
3,4,고른햇살,4.4,없음,성북구 개운사길 14
4,5,삼성통닭,4.0,첫째 셋째 일요일,성북구 고려대로24길 60


In [ ]:
def get_table_names(conn):
    """Return a list of table names."""
    table_names = []
    tables = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
    for table in tables.fetchall():
        table_names.append(table[0])
    # del table_names[0]
    return table_names


def get_column_names(conn, table_name):
    """Return a list of column names."""
    column_names = []
    columns = conn.execute(f"PRAGMA table_info('{table_name}');").fetchall()
    for col in columns:
        column_names.append(col[1])
    return column_names


def get_database_info(conn):
    """Return a list of dicts containing the table name and columns for each table in the database."""
    table_dicts = []
    for table_name in get_table_names(conn):
        columns_names = get_column_names(conn, table_name)
        table_dicts.append({"table_name": table_name, "column_names": columns_names})
    return table_dicts

In [ ]:
print(get_database_info(conn))

[{'table_name': 'anam_food', 'column_names': ['id', 'name', 'rating', 'closed_day', 'address']}]


In [ ]:
database_schema_dict = get_database_info(conn)
database_schema_string = "\n".join(
    [
        f"Table: {table['table_name']}\nColumns: {', '.join(table['column_names'])}"
        for table in database_schema_dict
    ]
)

In [ ]:
import json

def ask_database(query):
    """Function to query SQLite database with a provided SQL query."""
    try:
        results = str(conn.execute(query).fetchall())
    except Exception as e:
        results = f"query failed with error: {e}"
    return results

def execute_function_call(message):
    if message.tool_calls[0].function.name == "ask_database":
        query = json.loads(message.tool_calls[0].function.arguments)["query"]
        results = ask_database(conn, query)
    else:
        results = f"Error: function {message.tool_calls[0].function.name} does not exist"
    return results

In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "ask_database",
            "description": "\
            Use this function to answer user questions about restaurants in 안암.\
            Input should be a fully formed SQL query.\
            \
            ",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": f"""
                                SQL query extracting info to answer the user's question.
                                SQL should be written using this database schema:
                                {database_schema_string}
                                The query should be returned in plain text, not in JSON.
                                """,
                    }
                },
                "required": ["query"],
            },
        }
    }
]

In [ ]:
from openai import OpenAI
API_KEY = ""
client = OpenAI(api_key=API_KEY)

messages = []
messages.append({
    "role": "system",
    "content": "너는 안암 지역의 음식점에 대한 답변 챗봇이야.\
    너에게는 사전에 저장된 데이터베이스를 서치할 수 있는 Tool이 있어.\
    데이터베이스의 변수에 대한 설명은 다음과 같아.\
    -name: 음식점의 이름\
    -rating: 음식점의 평점에 대한 정보, 숫자가 높을 수록 높은 평점이다.\
    -closed_day: 음식점의 휴무일에 대한 정보\
    -address: 음식점의 주소\
    유저의 요청에 맞추어 알맞은 함수를 사용해줘. 답변은 친절하되 이모지를 사용해줘."
    })
prompt = "가장 평점이 높은 안암의 음식점은?"


response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=messages,
            tools=tools
            )
response


ChatCompletion(id='chatcmpl-96WMRdEhN70zThAquVakzhoRtqh1i', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_6miuw2SHE6AnWqTfxaOdnni9', function=Function(arguments='{"query":"SELECT name, rating, closed_day, address FROM anam_food"}', name='ask_database'), type='function')]))], created=1711341391, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_3bc1b5746c', usage=CompletionUsage(completion_tokens=26, prompt_tokens=325, total_tokens=351))

In [ ]:
finish_reason = response.choices[0].finish_reason
if finish_reason == "tool_calls":
  response_message = response.choices[0].message
response_message

ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_6miuw2SHE6AnWqTfxaOdnni9', function=Function(arguments='{"query":"SELECT name, rating, closed_day, address FROM anam_food"}', name='ask_database'), type='function')])

In [ ]:
function_name = response_message.tool_calls[0].function.name
function_name

'ask_database'

In [ ]:
arguments = json.loads(response_message.tool_calls[0].function.arguments)
arguments

{'query': 'SELECT name, rating, closed_day, address FROM anam_food'}

In [ ]:
function_response = eval(f"{function_name}(**arguments)")

function_response

"[('철남', 3.6, '없음', '성북구 안암로 73-7'), ('용초수', 4.3, '일요일', '성북구 고려대로27길 20'), ('이공김밥', 4.1, '일요일', '성북구 안암로 61-6'), ('고른햇살', 4.4, '없음', '성북구 개운사길 14'), ('삼성통닭', 4.0, '첫째 셋째 일요일', '성북구 고려대로24길 60')]"

In [ ]:
prompt = "가장 평점이 높은 안암의 음식점은?"
response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
	    {"role": "user", "content": prompt},
	    {"role": "function", "name": function_name, "content": function_response}
	 ],
    tools=tools
)
response

ChatCompletion(id='chatcmpl-96WMqOIKWotqXbrs7NJzJbRp1Mzfo', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_WEiVLaGZQZcUWfZU4DiyvBHp', function=Function(arguments='{"query":"SELECT name FROM anam_food ORDER BY rating DESC LIMIT 1;"}', name='ask_database'), type='function')]))], created=1711341416, model='gpt-3.5-turbo-0125', object='chat.completion', system_fingerprint='fp_3bc1b5746c', usage=CompletionUsage(completion_tokens=27, prompt_tokens=318, total_tokens=345))

In [ ]:
print(response.choices[0].message.content)

None


In [ ]:
%%writefile app.py
import streamlit as st
from openai import OpenAI

API_KEY = ""


st.title('날씨봇')
st.write("Hello ☁️🐯KAIROS. 궁금한 날씨 정보를 물어보세요")

if 'name' not in st.session_state:
    st.session_state.name = '유저이름'

st.header(st.session_state.name + '님 안녕하세요')
new_name = st.text_input(label='이름', value=st.session_state.name, placeholder='이름을 입력하세요', key='name')

client = OpenAI(api_key='(본인 API KEY)')

if "openai_model" not in st.session_state:
    st.session_state.openai_model = "(GPT 모델명)"

# Initialize chat history
if "messages" not in st.session_state:
    st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

# Accept user input for chat
if prompt := st.chat_input("What is up?"):
    # Add user message to chat history
    st.session_state.messages.append({"role": "user", "content": prompt})

    # Display user message in chat message container
    with st.chat_message("user"):
        st.markdown(prompt)

    # Display assistant response in chat message container
    with st.chat_message("assistant"):
        message_placeholder = st.empty()
        full_response = ""

        for response in client.chat.completions.create(
            model=st.session_state["openai_model"],
            messages=[{"role": "system", "content": prompt}],
            stream=True,
        ):
            full_response += (response.choices[0].delta.content or "")
            message_placeholder.markdown(full_response + "▌")
        message_placeholder.markdown(full_response)
    st.session_state.messages.append({"role": "assistant", "content": full_response})